In [1]:
from sklearn import preprocessing
import numpy as np
import pandas as pd
from collections import defaultdict

In [2]:
fields = ['package','forks','stars','so_tag_count','so_question_count','search_results']
DF = pd.read_csv('../output/distributed_computing_data.csv', usecols=fields)

In [3]:
#scale the values, such that for each metric: mean = 0 and std = 1
metrics = ['forks','stars','so_tag_count','so_question_count','search_results']
scaled_DF = pd.DataFrame(columns=metrics)

for metric in metrics:
    mean = DF[metric].mean()
    std = DF[metric].std()
    scaled_DF[metric] = DF[metric].apply(lambda x: (x-mean)/std)

scaled_DF['package']= DF['package']

In [4]:
#merge forks and stars to one GitHub metric, Tags and Questions to Stack Overflow
final_DF = pd.DataFrame(columns=['Package', 'Rank', 'Overall', 'Github', 'Stack Overflow', 'Search Results'])
final_DF['Package'] = scaled_DF['package']
final_DF['Github'] = scaled_DF[['forks', 'stars']].mean(axis=1)
final_DF['Stack Overflow'] = scaled_DF[['so_tag_count', 'so_question_count']].mean(axis=1)
final_DF['Search Results'] = scaled_DF['search_results']
final_DF['Overall'] = final_DF[['Github','Stack Overflow','Search Results']].sum(axis=1)
final_DF['Rank'] = final_DF['Overall'].rank(ascending=0).astype(int)

In [5]:
final_DF.sort_values(['Overall'], axis=0, ascending=False, inplace=True)
final_DF = final_DF.reindex_axis(['Package','Rank','Overall','Github','Stack Overflow','Search Results'], axis=1)
final_DF.style.set_table_styles([
    {'selector': '.row_heading, .blank', 'props': [('display', 'none;')]}
])

In [6]:
final_DF.to_csv('../output/DC_packages_final_Rankings.csv', index = False)